In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cmath as mt
from matplotlib import colors
import pickle
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.cross_validation
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
%matplotlib inline
import matplotlib as mpl
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords

In [1]:
#data = pd.read_pickle('..\output\data.pkl')
def download_data(n_directory,name_file):
    path = '..\output\X' + str(n_directory) + '\\'
    path_name_file = str(path) + str(name_file) + '.pkl'
    file = pd.read_pickle(path_name_file)
    return file 

def upload_data(N,file,name_file):
    path = '..\output\X' + str(N) + '\\'
    path_name_file_plk = str(path) + str(name_file) + '.pkl'
    path_name_file_csv = str(path) + str(name_file) + '.csv'    
    file.to_pickle(path_name_file_plk)
    file.to_csv(path_name_file_csv, sep='\t', encoding='utf-8')
    return path_name_file_plk 

def linear_regression_parameter(X, test_SIZE):
# Use only one feature
    X_input = X[['lenght_utterance','distance_to_end','median_duration']]
    Y_output = X['duration']

# Split the data into training/testing sets
# Split the targets into training/testing sets

    if test_SIZE == 0:
        X_train = X_input
        Y_train = Y_output
    else:    
        X_train, X_test, Y_train, Y_test = sklearn.cross_validation.train_test_split(X_input, Y_output, test_size = test_SIZE, random_state = 5)


# Create linear regression object
    regr = linear_model.LinearRegression()

# Train the model using the training sets
    regr.fit(X_train, Y_train)
    parameters = pd.DataFrame(list(zip(X_input.columns,regr.coef_)), columns= ['features','estimate_coefficients'])
    print(parameters)

    if test_SIZE == 0:
        return regr,parameters, X_train, Y_train
    else:
        
    
    
    # Make predictions using the testing set
        Y_pred = regr.predict(X_test) 

# The coefficients
# The mean squared error
        print("\nMean squared error: %.3f" % mean_squared_error(Y_test, Y_pred))
# Explained variance score: 1 is perfect prediction
        print('\nVariance score: %.2f' % r2_score(Y_test, Y_pred))

# Plot outputs
        Y_test=pd.DataFrame(Y_test)
        result = pd.concat([ Y_test, X_test], axis=1)
        index_selected = result.index.values
        df = pd.DataFrame({'index': index_selected, 'predicted_duration': Y_pred})
        df = df.set_index('index')
        result = pd.concat([ df, result], axis=1)
        result['distance_to_end'] = pd.to_numeric(result['distance_to_end'], errors='coerce').fillna(0)
        result['lenght_utterance'] = pd.to_numeric(result['lenght_utterance'], errors='coerce').fillna(0)
    
        result.plot(x = 'distance_to_end', y = 'predicted_duration', kind = 'scatter', grid=False,  alpha = 0.15,color='black', figsize= (15,10))
        result.plot(x = 'distance_to_end', y = 'duration', kind = 'scatter', grid=False,  alpha = 0.15,color='blue', figsize= (15,10))
        plt.xlabel("Distance utterance to the end (word)")
        plt.ylabel("Duration(s) ")
        plt.title("Expected & real duration vs distance utterance to the end ")
        plt.show()
    
        return regr,parameters, result, X_train, Y_train


In [2]:
def speech_rate_computation_2(X_processed, id_conv, id_speaker, word_to_exclude, word_trash):
    
    X_filtered_expected = X_processed[(~X_processed['word'].isin(word_to_exclude))]
    X_filtered_expected = X_filtered_expected[(X_filtered_expected['ID_conversation'] == id_conv) & (X_filtered_expected['ID_speaker'] == id_speaker)]
    
    X_filtered_real = X_processed[(~X_processed['word'].isin(word_trash))]
    X_filtered_real = X_filtered_real[(X_filtered_real['ID_conversation'] == id_conv) & (X_filtered_real['ID_speaker'] == id_speaker)]
    
    id_utterance = X_filtered_real['ID_utterance'].unique()
    for indice_id in id_utterance:
        X_filtered_utterance = X_filtered_real[X_filtered_real['ID_utterance'] == indice_id]
    
        codition_init = True
        condition_dataframe_not_empty = True
        while codition_init & condition_dataframe_not_empty:
            if X_filtered_utterance['word'].iloc[0] in word_to_exclude:
                indici = X_filtered_utterance.index
                c = indici[0]
                X_filtered_real.drop(c, inplace=True)
                X_filtered_utterance.drop(c, inplace=True)
                condition_dataframe_not_empty = ~X_filtered_utterance.empty
            else:
                codition_init = False
            
        codition_init = True
        condition_dataframe_not_empty = ~X_filtered_utterance.empty
        while codition_init & condition_dataframe_not_empty:
            if X_filtered_utterance['word'].iloc[-1] in word_to_exclude:
                indici = X_filtered_utterance.index
                c = indici[-1]
                X_filtered_real.drop(c, inplace=True)
                X_filtered_utterance.drop(c, inplace=True)
                condition_dataframe_not_empty = ~X_filtered_utterance.empty
            else:
                codition_init = False       
       
    real_duration = X_filtered_real.groupby(['ID_utterance'])['duration'].sum()
   # print(real_duration)
    expected_duration = X_filtered_expected.groupby(['ID_utterance'])['expected_duration'].sum()
   # print(expected_duration)
    pointwise_speech_rate = real_duration/expected_duration

    mean_log_pointwise_speech_rate = (np.log(pointwise_speech_rate)).mean() 
    mean_pointwise_speech_rate = pointwise_speech_rate.mean()
    
    real_duration_time = X_filtered_real.groupby(['ID_utterance'])['end_time'].max() - X_filtered_real.groupby(['ID_utterance'])['start_time'].min()
    t = real_duration_time.mean() + X_filtered_real.groupby(['ID_utterance'])['start_time'].min()
    

   # return [X_filtered_real, X_filtered_expected, pointwise_speech_rate]
    return [id_conv, mean_log_pointwise_speech_rate, pointwise_speech_rate, t, X_filtered_real, X_filtered_expected]  

In [3]:
def analyze_conv(id_conv,N_degree_fit, n_point_interpolation,n_window, word_to_exclude,word_trash, X_processed, speaker_speech_rate):

    [id_conv_A, mean_log_pointwise_speech_rate_A, pointwise_speech_rate_A, t_A, X_filtered_real_A, X_filtered_expected_A] = speech_rate_computation_2(X_processed, str(id_conv), 'A', word_to_exclude, word_trash)
    [id_conv_B, mean_log_pointwise_speech_rate_B, pointwise_speech_rate_B, t_B, X_filtered_real_B, X_filtered_expected_B] = speech_rate_computation_2(X_processed, str(id_conv), 'B', word_to_exclude, word_trash)
    fig = plt.figure(figsize=(12,7))
    ax1 = fig.add_subplot(111)
    pointwise_speech_rate_A.index = range(len(pointwise_speech_rate_A))
    pointwise_speech_rate_B.index = range(len(pointwise_speech_rate_B))


    pointwise_speech_rate_B_v = pointwise_speech_rate_B.as_matrix()
    t_B_v = t_B.as_matrix()
    zB = np.polyfit(t_B_v, pointwise_speech_rate_B_v, N_degree_fit)
    pB = np.poly1d(zB)
    x_B = np.linspace(t_B_v[0], t_B_v[-1], n_point_interpolation)


    pointwise_speech_rate_A_v = pointwise_speech_rate_A.as_matrix()
    t_A_v = t_A.as_matrix()
    zA = np.polyfit(t_A_v, pointwise_speech_rate_A_v, N_degree_fit)
    pA = np.poly1d(zA)
    x_A = np.linspace(t_A_v[0], t_A_v[-1], n_point_interpolation)
    #y_new = f(x_new)
    ax1.plot(x_A,pA(x_A), c='b',linestyle = '--', label='speaker A smooth')
    ax1.plot(x_B,pB(x_B), c='r',linestyle = '--', label='speaker B smooth')

    ax1.scatter(t_A, pointwise_speech_rate_A,linewidths=0.01,  c='b', label='speaker A')
    ax1.scatter(t_B, pointwise_speech_rate_B,linewidths=0.01,  c='r', label='speaker B')
    ax1.plot(t_A, pointwise_speech_rate_A,linewidth=0.5,  c='b')
    ax1.plot(t_B, pointwise_speech_rate_B,linewidth=0.5, c='r')
    plt.legend(loc='upper left');
    plt.suptitle('Speech rate vs time, conversation %s' %id_conv, fontsize=20)
    plt.xlabel('t(s)', fontsize=15)
    plt.ylabel('speech rate', fontsize=12)
    plt.show()

    yA = pA(x_A)
    xA = x_A
    derA = - np.diff(yA) / np.diff(xA)

    yB = pB(x_B)
    xB = x_B
    derB = - np.diff(yB) / np.diff(xB)


#    fig1 = plt.figure(figsize=(12,7))
#    ax2 = fig1.add_subplot(111)

#    ax2.plot(xA[0:-1:], derA,  c='b', label='speaker A')
#    ax2.plot(xB[0:-1:], derB, c='r', label='speaker B')

#    plt.legend(loc='upper left');
#    plt.suptitle('Derivative Speech rate vs time, conversation %s' %id_conv, fontsize=20)
#    plt.xlabel('t(s)', fontsize=15)
#    plt.ylabel('speech rate', fontsize=12)
#    plt.show()

    rollingB = pointwise_speech_rate_B.rolling(window=n_window)
    rolling_meanB = rollingB.mean()
    rollingA = pointwise_speech_rate_A.rolling(window=n_window)
    rolling_meanA = rollingA.mean()


    fig2 = plt.figure(figsize=(12,7))
    ax3 = fig2.add_subplot(111)

    ax3.plot(t_B_v,rolling_meanB,linestyle = '--',  c='r')

    ax3.plot(t_A_v,rolling_meanA,linestyle = '--',  c='b')


    ax3.scatter(t_A_v, rolling_meanA,linewidths=0.01,  c='b', label='Speaker')
    ax3.scatter(t_B_v, rolling_meanB,linewidths=0.01,  c='r', label='Interlocutor')

    rolling_meanB_noNan = rolling_meanB[~np.isnan(rolling_meanB)]
    zB_average = np.polyfit(t_B_v[(n_window-1):], rolling_meanB_noNan, N_degree_fit)
    pB_average = np.poly1d(zB_average)
    x_B = np.linspace(t_B_v[(n_window-1)], t_B_v[-1], n_point_interpolation)

    rolling_meanA_noNan = rolling_meanA[~np.isnan(rolling_meanA)]
    zA_average = np.polyfit(t_A_v[(n_window-1):], rolling_meanA_noNan, N_degree_fit)
    pA_average = np.poly1d(zA_average)
    x_A = np.linspace(t_A_v[(n_window-1)], t_A_v[-1], n_point_interpolation)

    ax3.plot(x_B,pB_average(x_B), c='r',linestyle = '-', label='Interlocutor smooth')
    ax3.plot(x_A,pA_average(x_A), c='b',linestyle = '-', label='Speaker smooth')
    
    
    baseline_A = speaker_speech_rate[(speaker_speech_rate['id_conv'] == id_conv) & (speaker_speech_rate['id_caller'] == 'A')].baseline_no_log_S.item()
    baseline_B = speaker_speech_rate[(speaker_speech_rate['id_conv'] == id_conv) & (speaker_speech_rate['id_caller'] == 'A')].baseline_no_log_I.item()
    
    media_A = pointwise_speech_rate_A.mean() 
    x_A = [t_A.min(), t_A.max()]
    y_A = [media_A, media_A]
    ax3.plot(x_A,y_A, label='Speaker Mean')
    y_baseline_A = [baseline_A,baseline_A]

    ax3.plot(x_A,y_baseline_A, c = 'navy',label='Speaker Baseline')

    media_B = pointwise_speech_rate_B.mean() 
    x_B = [t_B.min(), t_B.max()]
    y_B = [media_B, media_B]
    y_baseline_B = [baseline_B,baseline_B]
    ax3.plot(x_B,y_B,c='hotpink', label='Interlocutor Mean')
    ax3.plot(x_B,y_baseline_B,c='purple', label='Interlocutor Baseline')
    ax3.set_position([0.08,0.08,0.75,0.8])
        
    
    plt.suptitle('Speech Rate vs time, conversation %s' %id_conv, fontsize=20)
    plt.xlabel('t(s)', fontsize=15)
    plt.ylabel('speech rate', fontsize=12)
    plt.legend()
    ax3.legend(loc='center left', bbox_to_anchor=(1, 0.8))
    
    name_fig = './graphics/' + 'ID_' + str(id_conv) + '.png'
    fig2.savefig(name_fig)
    plt.show()
    

    
#_______________________

In [1]:
def diff_speech_rate(N,X_processed,id_conv, word_to_exclude, word_trash):
    [id_conv_A, mean_log_pointwise_speech_rate_A, pointwise_speech_rate_A, t_A, X_filtered_real_A, X_filtered_expected_A ] = speech_rate_computation_2(X_processed, str(id_conv), 'A', word_to_exclude, word_trash)
    [id_conv_B, mean_log_pointwise_speech_rate_B, pointwise_speech_rate_B, t_B, X_filtered_real_B, X_filtered_expected_B] = speech_rate_computation_2(X_processed, str(id_conv), 'B', word_to_exclude, word_trash)
    dA = {'speech_rate': pointwise_speech_rate_A, 'time': t_A}
    df_A = pd.DataFrame(data=dA)
    dB = {'speech_rate': pointwise_speech_rate_B, 'time': t_B}
    df_B = pd.DataFrame(data=dB)
    inf = min(t_A.min(), t_B.min())
    sup = max(t_A.max(), t_B.max())
    incr = (sup - inf)/N
    limits = [(incr*i + inf)  for i in range(N + 1)]
    sp_DA=[]
    sp_DB=[]
    time_DA = []
    time_DB = []
    time_vector_A = []
    srate_vector_A = []
    time_vector_B = []
    srate_vector_B = []
    

    for j in range(N):

        df_A_filterd = df_A[(df_A['time'] < limits[j+1]) & (df_A['time'] > limits[j])]
        df_B_filterd = df_B[(df_B['time'] < limits[j+1]) & (df_B['time'] > limits[j])]
        average_sp_dA = df_A_filterd['speech_rate'].mean()
        average_sp_dB = df_B_filterd['speech_rate'].mean()
        average_time_dA = df_A_filterd['time'].mean()
        average_time_dB = df_B_filterd['time'].mean()

#####################################################

        time_vector_A.append(df_A_filterd['time'].values)
        srate_vector_A.append(df_A_filterd['speech_rate'].values)
        time_vector_B.append(df_B_filterd['time'].values)
        srate_vector_B.append(df_B_filterd['speech_rate'].values)
            
    
        sp_DA.append(average_sp_dA)
        sp_DB.append(average_sp_dB)
        time_DA.append(average_time_dA)
        time_DB.append(average_time_dB)

    diff_sp = [x - y for x, y in zip(sp_DA, sp_DB)]   
    time_average = [x / 2 for x in limits]
    duration = sup - inf
    return [diff_sp,sp_DA, sp_DB, t_A.min(), t_A.max(), t_B.min(), t_B.max(), duration, time_vector_A, srate_vector_A, time_vector_B, srate_vector_B]

# create file for the visualization data of Antoine Gaboin
id_conv_list = X['ID_conversation'].unique()
for id_conv in id_conv_list:
    for id_speaker in ['A', 'B']:
        [id_conv, mean_log_pointwise_speech_rate, pointwise_speech_rate, t, X_filtered_real,
        X_filtered_expected] =speech_rate_computation_2(X, str(id_conv), id_speaker, word_to_exclude, word_trash)
        pointwise_speech_rate = pd.DataFrame(pointwise_speech_rate)
        t = pd.DataFrame(t)
        speech_rate_conv = pd.concat([ pointwise_speech_rate, t], axis=1)
        speech_rate_conv.columns = ['y', 't']
        name_file = 'SW_' + str(id_conv) + '_speech_rate_' + id_speaker
        upload_data(11,speech_rate_conv,name_file)

In [2]:
import time, sys

# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 20 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rPercent: [{0}] {1}% {2}".format( "="*block + " "*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()